In [1]:
import pandas as pd 
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer

c:\Users\82106\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
# SBERT 모델 로드
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

c:\Users\82106\AppData\Local\Programs\Python\Python310\lib\site-packages\sentence_transformers\SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


In [9]:
file_path = r"C:\Users\82106\Desktop\데이콘 한솔데크\data\train.csv"
train = pd.read_csv(file_path)

file_path = r"C:\Users\82106\Desktop\데이콘 한솔데크\data\test.csv"
test = pd.read_csv(file_path)

file_path = r"C:\Users\82106\Desktop\데이콘 한솔데크\data\sample_submission.csv"
sample = pd.read_csv(file_path)

In [10]:
# 인적사고, 작업프로세스, 부위를 기준으로 그룹화
grouped = train.groupby(["인적사고", "작업프로세스"])

res = {}
cosine_res = []
for name, group in tqdm(grouped):
    plan = group["재발방지대책 및 향후조치계획"]
    vectors = np.stack(plan.apply(model.encode).to_numpy())
    similarity = cosine_similarity(vectors, vectors)    
    cosine_res += similarity[similarity.mean(axis=1).argmax()].tolist()
    res[name] = plan.iloc[similarity.mean(axis=1).argmax()]

  0%|          | 0/814 [00:00<?, ?it/s]

In [11]:
# 히스토그램 생성
arr = cosine_res
bins = np.arange(0, 1.1, 0.1)  # 0.0 ~ 1.0을 0.1 간격으로 나눔
hist, bin_edges = np.histogram(arr, bins=bins)

# 히스토그램 출력
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")

Range 0.0 - 0.1: 1개
Range 0.1 - 0.2: 25개
Range 0.2 - 0.3: 180개
Range 0.3 - 0.4: 687개
Range 0.4 - 0.5: 1810개
Range 0.5 - 0.6: 4115개
Range 0.6 - 0.7: 7234개
Range 0.7 - 0.8: 6677개
Range 0.8 - 0.9: 1739개
Range 0.9 - 1.0: 565개


In [12]:
res

{('감전',
  '기타'): '운행선 인접공사 시 전차선 가압 여부 확인 철저, 선로 내 지장물 취급 시 안전사항 확인 철저, 사전 타분야 간 작업 협의 철저.',
 ('감전',
  '도장작업'): '공도구 및 리드선 점검, 누전차단기 없는 것들의 폐기, 전기기계기구 및 외함 접지 실시, 호우 후 고인 빗물 제거.',
 ('감전', '벌목작업'): '안전교육 철저 및 주의사항 안내와 안전관리 철저를 포함한 재발 방지 대책과 향후 조치 계획.',
 ('감전', '보수 및 교체작업'): '작업자 안전교육 철저와 작업전 안전회의 시행을 통한 작업자 부주의로 인한 안전사고 방지.',
 ('감전', '부설 및 다짐작업'): '작업구간 내 간섭되는 전선 전수 조사 및 사고 예방대책 시행 후 작업 시행.',
 ('감전', '설비작업'): '현장대리인 및 공사감리자 안전교육 및 후속점검 예정.',
 ('감전', '설치작업'): '사고사례 전파 및 안전교육을 통한 재발 방지 대책과 향후 조치 계획.',
 ('감전', '용접작업'): '작업 전 유해위험건설기계의 고장 여부 확인 및 정기 점검 실시.',
 ('감전',
  '운반작업'): '위험 구간 및 장소의 장비 사용 금지와 위험 요소에 대한 사전 안전 조치 강화, 안전 관리자의 교육 및 책임 강화, 작업 근로자의 안전 교육 및 개인 보호구 착용 의무 강화, 전문 인력 배치, 사고 현장에 대한 한시적 즉각 공사 정지 명령과 관련 사항 점검, 해당 과실 관련 주체의 처벌 및 기타 민형사상 행정 처분, 향후 재발 방지 계획 수렴 후 적법성 인정 시 공사 정지 명령 철회.',
 ('감전',
  '이동'): '전주 이설 후 작업 지시와 안전에 취약한 공종 작업 시 안전관리자 및 공사감독자 확인 후 작업지시의 통합적 관리 방안.',
 ('감전',
  '적재작업'): '안전교육 실시와 현장 안전관리 철저를 통한 재발 방지 대책 및 정밀검사 결과에 따른 입원 요양 중 경과에 따른 후속 조치 계획.',
 ('감전',
  '전기작업'): '

In [13]:
# 대표 대책 벡터화
res_v = {}
for k,v in res.items():
    res_v[k] = model.encode(v)

# 테스트 데이터 반영
for i in range(len(test)):
    accident = test.loc[i, "인적사고"]
    process = test.loc[i, "작업프로세스"]
    body_part = test.loc[i, "부위"]
    sample.loc[i, "재발방지대책 및 향후조치계획"] = res.get((accident, process, body_part), "안전교육 실시")
    sample.iloc[i, 2:] = res_v.get((accident, process, body_part), np.zeros(768))

In [14]:
sample.to_csv("C:/Users/82106/Desktop/데이콘 한솔데크/data/result/Grouping.csv", index=False, encoding='utf-8-sig')